In [ ]:
from ._anvil_designer import CNNTemplate
from anvil import *
import anvil.server
import anvil.js
import time

ROW_COUNT = 6
COLUMN_COUNT = 7

class CNN(CNNTemplate):
    def __init__(self, **properties):
        # Initialize Form components
        self.init_components(**properties)

        # Set canvas size if not using Anvil IDE’s size settings
        self.canvas_1.width = 580
        self.canvas_1.height = 580

        # Create a 6x7 board (all zeros)
        self.board = [[0] * COLUMN_COUNT for _ in range(ROW_COUNT)]
        self.turn = 1  # Player 1 (red) starts
        self.selected_model = "cnn"  # For AI move calls

        # Set the initial label to show it's the user's move
        self.turn_label.text = "Your move 🔴"
        # Initialize sound state (off by default)
        self.is_music_playing = False
        self.sound_button.text = "Sound Off"  # Default text for the sound toggle button
        # Draw the initial (empty) board
        self.draw_board()


    # ------------------------------------------------------------------
    # ENABLE/DISABLE SPOT BUTTONS
    # ------------------------------------------------------------------
    def set_spot_buttons_enabled(self, enable):
        """
        Enable or disable all 7 spot (column) buttons.
        """
        self.Spot1.enabled = enable
        self.Spot2.enabled = enable
        self.Spot3.enabled = enable
        self.Spot4.enabled = enable
        self.Spot5.enabled = enable
        self.Spot6.enabled = enable
        self.Spot7.enabled = enable

    # ------------------------------------------------------------------
    # DROP PIECE LOCALLY
    # ------------------------------------------------------------------
    def drop_piece_locally(self, board, col, player_piece):
        """
        Places 'player_piece' (1 or 2) in the lowest available
        row of 'col' in 'board'. Returns the row where the piece
        landed, or None if the column is full.
        """
        for row in range(ROW_COUNT - 1, -1, -1):
            if board[row][col] == 0:
                board[row][col] = player_piece
                return row
        return None  # Column is full

    # ------------------------------------------------------------------
    # COLUMN CLICK -> USER & AI TURN
    # ------------------------------------------------------------------
    def on_column_click(self, col):
        """
        Called when user clicks one of the 7 columns.
        This handles the user move, checks for winner,
        then handles the AI move if needed.
        """
        # If buttons are already disabled, do nothing (prevents double-click spamming)
        if not self.Spot1.enabled:
            return

        # Disable the column buttons so the user cannot click again immediately
        self.set_spot_buttons_enabled(False)

        # The label should already read "Your move 🔴" (set in constructor),
        # but if you want to ensure it, you can uncomment this:
        # self.turn_label.text = "Your move 🔴"

        # 1) User drops piece in the chosen column
        new_piece_row = self.drop_piece_locally(self.board, col, 1)  # 1 = user
        if new_piece_row is None:
            alert("Column is full. Choose another column!")
            # Re-enable buttons since user can choose another column
            self.set_spot_buttons_enabled(True)
            return
        # 2) Animate the user piece falling
        board_snapshot = [r[:] for r in self.board]
        self.board[new_piece_row][col] = 0
        self.animate_fall(col, new_piece_row, self.board, player_piece=1)
        # Now restore the piece
        self.board[new_piece_row][col] = 1
        self.draw_board()

        # 3) Check if user won
        if anvil.server.call_s('check_winner_server', self.board, 1):
            self.play_sound("win.mp3")
            alert("You Won!")
            self.reset_game()
            return  # Don't re-enable; game ended

        # 4) Check draw
        if self.is_board_full():
            self.play_sound("game_over.mp3")
            alert("It's a draw!")
            self.reset_game()
            return  # Game ended

        # 5) Now it's AI's turn
        self.turn_label.text = "AI Move 🟡"

        # 6) Ask server for AI's move
        result = anvil.server.call_s('get_ai_move', self.board, self.selected_model)
        if isinstance(result, dict) and 'error' in result:
            alert(result['error'])
            # Re-enable buttons so user can try again
            self.turn_label.text = "Your move 🔴"
            self.set_spot_buttons_enabled(True)
            return
        ai_col = result  # column index 0..6

        # 7) Drop AI piece locally
        ai_new_piece_row = self.drop_piece_locally(self.board, ai_col, 2)  # 2 = AI
        if ai_new_piece_row is None:
            alert("AI tried to play in a full column. Something's off.")
            self.turn_label.text = "Your move 🔴"
            self.set_spot_buttons_enabled(True)
            return

        # 8) Animate the AI piece
        self.board[ai_new_piece_row][ai_col] = 0
        self.animate_fall(ai_col, ai_new_piece_row, self.board, player_piece=2)
        self.board[ai_new_piece_row][ai_col] = 2
        self.draw_board()

        # 9) Check if AI won
        if anvil.server.call_s('check_winner_server', self.board, 2):
            self.play_sound("game_over.mp3")
            alert("Aha! Try Again")
            self.reset_game()
            return  # Game ended

        # 10) Check draw
        if self.is_board_full():
            self.play_sound("game_over.mp3")
            alert("It's a draw!")
            self.reset_game()
            return

        # If we reach here, the game continues, so switch label back to user
        self.turn_label.text = "Your move 🔴"

        # Re-enable buttons so user can make the next move
        self.set_spot_buttons_enabled(True)

    # ------------------------------------------------------------------
    # ANIMATE FALL
    # ------------------------------------------------------------------
    def animate_fall(self, col, final_row, board_snapshot, player_piece):
        """
        Show a piece 'falling' from top to the given row,
        but the board remains in its OLD state (board_snapshot)
        during the animation.
        """
        c = self.canvas_1
        width, height = c.width, c.height
        cell_width = width // COLUMN_COUNT
        cell_height = height // ROW_COUNT

        piece_color = "red" if player_piece == 1 else "yellow"
        x = col * cell_width + cell_width // 2
        target_y = final_row * cell_height + cell_height // 2
        radius = min(cell_width, cell_height) // 2 - 5

        step = max(cell_height // 10, 1)

        for y in range(0, target_y + 1, step):
            c.clear_rect(0, 0, width, height)
            self.draw_board(custom_board=board_snapshot)
            c.fill_style = piece_color
            c.begin_path()
            c.arc(x, y, radius, 0, 2 * 3.14159)
            c.close_path()
            c.fill()
            time.sleep(0.01)

    # ------------------------------------------------------------------
    # DRAW BOARD
    # ------------------------------------------------------------------
    def draw_board(self, custom_board=None):
        board_to_draw = custom_board if custom_board else self.board
        c = self.canvas_1
        width, height = c.width, c.height
        cell_width = width // COLUMN_COUNT
        cell_height = height // ROW_COUNT

        c.clear_rect(0, 0, width, height)

        # Background color
        c.fill_style = "#005235"  # Dark greenish
        c.fill_rect(0, 0, width, height)

        colors = {0: "white", 1: "red", 2: "yellow"}
        for row in range(ROW_COUNT):
            for col in range(COLUMN_COUNT):
                x = col * cell_width + (cell_width // 2)
                y = row * cell_height + (cell_height // 2)
                radius = min(cell_width, cell_height) // 2 - 5

                c.stroke_style = "black"
                c.begin_path()
                c.arc(x, y, radius, 0, 2 * 3.14159)
                c.close_path()
                c.stroke()

                value = board_to_draw[row][col]
                c.fill_style = colors[value]
                c.fill()

    # ------------------------------------------------------------------
    # CHECK IF BOARD IS FULL
    # ------------------------------------------------------------------
    def is_board_full(self):
        return all(0 not in row for row in self.board)

    # ------------------------------------------------------------------
    # RESET
    # ------------------------------------------------------------------
    def reset_game(self):
        """Reset the board to empty and turn to 1."""
        self.board = [[0] * COLUMN_COUNT for _ in range(ROW_COUNT)]
        self.turn = 1
        self.draw_board()
        # Re-enable columns for new game
        self.set_spot_buttons_enabled(True)
        # Reset label
        self.turn_label.text = "Your move 🔴"

    # ------------------------------------------------------------------
    # SPOT CLICK HANDLERS
    # ------------------------------------------------------------------
    def Spot1_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(0)

    def Spot2_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(1)

    def Spot3_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(2)

    def Spot4_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(3)

    def Spot5_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(4)

    def Spot6_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(5)

    def Spot7_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        self.on_column_click(6)

    # ------------------------------------------------------------------
    # OTHER BUTTONS (HOME, TRANSFORMER, RESTART, etc.)
    # ------------------------------------------------------------------
    def button_1_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        open_form('Home')

    def button_2_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        open_form('Home.Play_game.Transformer')
  
    def restart_click(self, **event_args):
        self.play_sound("button-3-214381.mp3")
        open_form('Home.Play_game.CNN')

    # ------------------------------------------------------------------
    # PLAY SOUND
    # ------------------------------------------------------------------
    def play_sound(self, sound_file):
        """
        Plays a sound using JavaScript's Audio API.
        We build a URL to the sound file in our theme assets,
        then create a JavaScript audio element to play it.
        """
        file_url = f"{anvil.server.get_app_origin()}/_/theme/{sound_file}"
        js_code = f"""
        var audio = new Audio("{file_url}");
        audio.play();
        """
        anvil.js.window.eval(js_code)

    def play_background_music_game(self, sound_file):
      """Plays looping background music using JavaScript without lag."""
      js_code = f"""
      if (!window.bgAudio) {{
          window.bgAudio = new Audio("{anvil.server.get_app_origin()}/_/theme/{sound_file}");
          window.bgAudio.loop = true;
          window.bgAudio.play();
      }} else {{
          if (window.bgAudio.paused) {{
              window.bgAudio.play();
          }}
      }}
      """
      anvil.js.window.eval(js_code)

    def stop_music(self):
      """Stops any currently playing background music without resetting it."""
      js_code = "if (window.bgAudio) { window.bgAudio.pause(); }"
      anvil.js.window.eval(js_code)
    
    def sound_game(self, **event_args):
      """Toggles sound on and off when the button is clicked."""
      if self.is_music_playing:
          self.stop_music()
          self.sound_button.text = "Sound Off"
      else:
          self.play_background_music_game("background_music.mp3")
          self.sound_button.text = "Sound On"
        # Toggle the state
      self.is_music_playing = not self.is_music_playing
